In [2]:
!pip install catboost
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from dateutil import parser
from collections import defaultdict
from copy import deepcopy
import random
import statistics
import math
import numpy as np
import ast
import statistics
import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from catboost import CatBoostClassifier

import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import precision_recall_fscore_support

In [4]:
%cd /content/drive/MyDrive/Colab_Notebooks/LDF

/content/drive/.shortcut-targets-by-id/10cLNvGYZ8jCz5UzVVMmUw1kONRPJ_red/LDF


# Help functions

## Data imputation

In [5]:
def data_imputation(X_train, X_test, imputation_method='mean'):
  # This function imputes the data based on the train set
  # and apply the imputed values to the test set

  if imputation_method == 'mean':

    # Calculate mean per column
    n = len(X_train[0])
    memo = [np.asfarray([0 for i in range(n)]), np.asfarray([0.0 for i in range(n)])] # [Count, Sum]

    for i in range(len(X_train)):
      for j in range(len(X_train[i])):
        if np.isnan(X_train[i][j]) != True:
          memo[0][j] += 1
          memo[1][j] += X_train[i][j]

    for j in range(n):
      memo[1][j] = memo[1][j]/memo[0][j] # ==> [Count, Mean]

    # Data imputation
    for i in range(len(X_train)):
      for j in range(len(X_train[i])):
        if np.isnan(X_train[i][j]) == True:
          X_train[i][j] = memo[1][j]

    for i in range(len(X_test)):
      for j in range(len(X_test[i])):
        if np.isnan(X_test[i][j]) == True:
          X_test[i][j] = memo[1][j]

  return X_train, X_test

## In-the-loop normalization

In [6]:
def data_normalization(X_train, X_test):
  # This function normalizes the data based on the train set
  # and apply the normalized values to the test set

  # Convert to float to get more precise results (yet empirically proved)
  X_train = np.asfarray(X_train)
  X_test = np.asfarray(X_test)

  # Memoization: [Minimums, Maximums]
  memo = deepcopy([X_train[0], X_train[0]])
  for row in X_train:
    for i in range(len(row)):
      if row[i] < memo[0][i]:
        memo[0][i] = row[i]
      if row[i] > memo[1][i]:
        memo[1][i] = row[i]

  # Normalization on train set
  for r in range(len(X_train)):
    for c in range(len(X_train[0])):
      if memo[1][c]-memo[0][c] == 0:
        X_train[r][c] = (X_train[r][c] - memo[0][c])/(1e-20)
      else:
        X_train[r][c] = (X_train[r][c] - memo[0][c])/(memo[1][c]-memo[0][c])

  # Normalization on test set
  for r in range(len(X_test)):
    for c in range(len(X_test[0])):
      if memo[1][c]-memo[0][c] == 0:
        X_test[r][c] = (X_test[r][c] - memo[0][c])/(1e-20)
      else:
        X_test[r][c] = (X_test[r][c] - memo[0][c])/(memo[1][c]-memo[0][c])
      # Rescale
      if X_test[r][c] > 1.0:
        X_test[r][c] = 1.0
      elif X_test[r][c] < 0.0:
        X_test[r][c] = 0.0

  return X_train, X_test

## Leave one person out

In [7]:
def leave_one_person_out(patient_ids):
  loo = dict()
  for patient_id in set(patient_ids):
    loo[patient_id] = None

  for patient_id in loo:
    train_indices, test_indices = [], []

    for i in range(len(patient_ids)):
      if patient_ids[i] == patient_id:
        test_indices.append(i)
      else:
        train_indices.append(i)

    loo[patient_id] = [train_indices, test_indices]

  return loo

## K-fold

In [8]:
def k_fold(patient_ids, number_of_folds = 5):
  patient_id_dict = dict()
  for patient_id in set(patient_ids):
    patient_id_dict[patient_id] = []

  for i in range(len(patient_ids)):
    patient_id = patient_ids[i]
    patient_id_dict[patient_id].append(i)

  kfold_dict = dict()
  i, k = 0, len(patient_id_dict.keys())//number_of_folds
  queue = list(patient_id_dict.keys())

  while queue:
    random.shuffle(queue)
    patient_id_test = [j for j in queue[0:k] ]
    patient_id_train = [j for j in set(patient_id_dict.keys())-set(patient_id_test) ]

    train_indices, test_indices = [], []
    for patient_id in patient_id_train:
      train_indices += patient_id_dict[patient_id]
    for patient_id in patient_id_test:
      test_indices += patient_id_dict[patient_id]
    kfold_dict[i] = [train_indices, test_indices]

    queue = queue[k:]
    i += 1

  return kfold_dict

## AUC plot

In [9]:
def AUC_plot(y, preds):
  figure, axis = plt.subplots(1, 2)

  # ROC AUC
  fpr, tpr, thresholds = roc_curve(y, preds, #pos_label=1
                                   )
  roc_auc = round(auc(fpr, tpr), 4)
  print('ROC AUC: ', roc_auc)
  #print('FPR: ', fpr, 'TPR: ', tpr) # For debugging
  axis[0].plot(fpr, tpr)
  axis[0].set_title('ROC AUC')
  axis[0].set_xlabel('FPR')
  axis[0].set_ylabel('TPR')

  # PR AUC
  precision, recall, thresholds = precision_recall_curve(y, preds, pos_label=1)
  pr_auc = round(auc(recall,precision), 4)
  print('PR AUC: ', pr_auc)
  #print('Recall: ', recall, 'Precision: ', precision) # For debugging
  axis[1].plot(recall, precision)
  axis[1].set_title('ROC PR')
  axis[1].set_xlabel('Recall')
  axis[1].yaxis.set_label_position("right")
  axis[1].set_ylabel('Precision')

  plt.show()

## One-hot encoding

In [10]:
def onehot_encoding(df):
  # Encode categorical variables and store the mapping
  label_encoder = LabelEncoder()
  encoded_values = {}

  for column in df.columns:
    if df[column].dtype == 'O' and column != '1. Volunteer full name':
      df[column] = label_encoder.fit_transform(df[column])
      encoded_values[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

      print(f'Feature: {column} \n', encoded_values[f'{column}'])

  return encoded_values, df

## MLP Binary

In [11]:
# Define the MLP architecture for binary classification
class MLPBinary(nn.Module):
  def __init__(self, input_size, hidden_size, dropout_prob):
    super(MLPBinary, self).__init__()
    self.layer_norm = nn.LayerNorm(input_size)
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(dropout_prob)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.layer_norm(x)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.dropout(out)
    out = self.fc3(out)
    out = self.sigmoid(out)
    return out


def MLP_training(X_train, X_test, y_train, y_test):
  # Data imputation
  X_train, X_test = data_imputation(X_train, X_test,
                                    imputation_method='mean')

  # Data normalization
  X_train, X_test = data_normalization(X_train, X_test)

  # Convert NumPy arrays to PyTorch tensors
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

  # Convert data to PyTorch tensors
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

  # Create DataLoader
  train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  # Initialize model, loss function, and optimizer
  model = MLPBinary(input_size, hidden_size, dropout_prob)
  criterion = nn.BCELoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  # Define variables to track the best model and its accuracy
  best_accuracy = 0
  best_model = None

  # Training the model
  for epoch in range(num_epochs):
    # Train
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels.unsqueeze(1))
      loss.backward()
      optimizer.step()

      # Evaluate on train set
      train_outputs = model(X_train_tensor)
      train_predicted = (train_outputs > 0.5).squeeze().int()
      train_accuracy = (train_predicted == y_train_tensor.int()).sum().item() / len(y_train_tensor)

    # Evaluate on test set
    with torch.no_grad():
      model.eval()
      test_outputs = model(X_test_tensor)
      test_predicted = (test_outputs > 0.5).squeeze().int()
      test_accuracy = (test_predicted == y_test_tensor.int()).sum().item() / len(y_test_tensor)

    # Print accuracy for each epoch
    print('Epoch [{}/{}], Train Accuracy: {:.4f}, Test Accuracy: {:.4f}'.format(epoch+1, num_epochs, train_accuracy, test_accuracy))

    # Save the best model
    if test_accuracy > best_accuracy:
      best_accuracy = test_accuracy
      best_model = model.state_dict()

  # Save the best model
  torch.save(best_model, 'best_model.pth')


def MLP_inference(X_test):
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
  model = MLPBinary(input_size, hidden_size, dropout_prob)
  model.load_state_dict(torch.load('best_model.pth'))
  model.eval()

  with torch.no_grad():
    y_test_predicted = model(X_test_tensor).squeeze()
    y_test_predicted = y_test_predicted.numpy()

  return y_test_predicted

## MLP Multiclass

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.layer_norm = nn.LayerNorm(input_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, int(hidden_size*5))
        self.fc3 = nn.Linear(int(hidden_size*5), num_classes)

    def forward(self, x):
        out = self.layer_norm(x)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out



# Top 10 features - Multiclass classification

In [13]:
df_rawdata = pd.read_excel('Data/20-05-2024_TextNorm.xlsx')
print(df_rawdata.dtypes)
df_rawdata

Type of data                                 object
Patient_ID                                   object
Kv100                                       float64
M                                           float64
δ                                           float64
T                                           float64
A365                                        float64
A460                                        float64
Anadn                                       float64
POM                                         float64
Ae                                          float64
An                                          float64
Am                                          float64
Ar                                          float64
Ac                                          float64
F_Ae                                        float64
F_An                                        float64
F_Am                                        float64
F_Ar                                        float64
F_Ac        

,Type of data,Patient_ID,Kv100,M,δ,T,A365,A460,Anadn,POM,...,Disease,Blood Pressure,Level of BP,Heart Rate,DASS_21,Stress,Anxiety,Depression,Hypertension,Non-wellbeing
0,Original,Minh Ngoc Nguyen,12.90,22.50,2.90,32.01,114.0,62.0,0.54,5.68,...,Migraine,Normal,0,95.0,NORMAL,0,0,0,0,0
1,Original,Minh Ngoc Nguyen,22.28,22.55,5.02,32.14,122.0,42.0,0.34,10.38,...,Migraine,Normal,0,95.0,NORMAL,0,0,0,0,0
2,Filter,Minh Ngoc Nguyen,11.10,22.85,2.54,32.04,114.0,62.0,0.54,7.54,...,Migraine,Normal,0,95.0,NORMAL,0,0,0,0,0
3,Filter,Minh Ngoc Nguyen,20.78,23.01,4.78,23.01,122.0,41.0,0.33,11.31,...,Migraine,Normal,0,95.0,NORMAL,0,0,0,0,0
4,Original,Hong Pham,20.76,4.39,0.91,24.44,132.0,93.0,0.70,0.91,...,Migraine,Normal,0,70.0,NORMAL,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Original,Giang Thi Sot,28.89,16.62,4.80,30.75,78.0,97.0,1.24,3.02,...,"Diabetes, MACE",Elevated,1,70.0,NORMAL,0,0,0,1,0
1074,Original,Nguyen Ba Tuyen,27.52,18.78,5.17,36.86,92.0,71.0,0.77,3.77,...,"Diabetes, MACE",Elevated,1,70.0,NORMAL,0,0,0,1,0
1075,Original,Nguyen Ba Tuyen,10.34,23.96,2.48,35.89,82.0,31.0,0.37,13.40,...,"Diabetes, MACE",Elevated,1,70.0,NORMAL,0,0,0,1,0
1076,Original,Mac Van Thang,26.75,24.51,6.56,35.07,53.0,40.0,0.77,7.49,...,MACE,Elevated,1,70.0,NORMAL,0,0,0,1,0


In [14]:
#print(df_rawdata['DASS_21'].unique()) # For debugging

# One-hot encoding
encoded_values, df_rawdata = onehot_encoding(df_rawdata)

Feature: Type of data 
 {'Filter': 0, 'Fragment 1': 1, 'Fragment 2': 2, 'Fragment 3': 3, 'Original': 4, 'fragment1': 5, 'fragment2': 6}
Feature: Patient_ID 
 {'A Amalraja': 0, 'Abdul Raheem': 1, 'Aisha Bibi': 2, 'Aleksandr Koviarov': 3, 'Alice': 4, 'Anh Tai': 5, 'Anh Toan': 6, 'Arooj Arooj': 7, 'Badmos': 8, 'Barandewte Rose': 9, 'Bui Thi Hue': 10, 'Bui Thi Phú': 11, 'Bùi Kim Dung': 12, 'Bùi Thuý Thi': 13, 'Bùi Xuân Khoa': 14, 'Caroline': 15, 'Chi Hao': 16, 'Cong Huan Hoang': 17, 'Dam Thi Thoa': 18, 'Daniel': 19, 'Dao Huy Co': 20, 'Dao Van Tuy': 21, 'Diana Galiakhmetova': 22, 'Dini': 23, 'Do Van Khai': 24, 'Doan Thi Chu': 25, 'Doan Xuan Dinh': 26, 'Duc Tran van': 27, 'Duong Thu Trang': 28, 'E Lieu': 29, 'E Nam': 30, 'E Phi': 31, 'E Thuy': 32, 'E Trung': 33, 'Em Son': 34, 'Emad Tandis': 35, 'Fanchesco': 36, 'Gauthami': 37, 'Geraldo Gomes De Luna Junior': 38, 'Giang Thi Sot': 39, 'Hani Kbashi': 40, 'Hien dang': 41, 'Hoang Thi Mien': 42, 'Hoang Thi Quynh Ngan': 43, 'Hoang Thi Sam': 44, 'Ho

In [15]:
input = ['BMI_index', 'Heart Rate', 'Age', 'Type of skins', 'Weight',
         'T', 'Height', 'A460', 'Anadn', 'M'
         ]
output = [#'Ethnicity', 'Left or Right', 'Sleep', 'Gender', 'Race',
          #'Smoking',
          #'Disease',
          'DASS_21',
          #'Stress', 'Anxiety', 'Depression', 'Hypertension', 'Non-wellbeing'
          ]

X_rawdata = df_rawdata[input].values
y_dass21 = df_rawdata['DASS_21'].values

patient_ids = list(df_rawdata['Patient_ID'])

## Leave-one-person-out

### Gradient Boosting

In [16]:
# Gradient Boosting
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

loo = leave_one_person_out(patient_ids)

for patient_id in list(loo.keys()):
  # Leave one out
  train_indices = np.array(loo[patient_id][0])
  test_indices = np.array(loo[patient_id][1])

  X_rawdata_train = X_rawdata[train_indices]
  X_rawdata_test = X_rawdata[test_indices]
  y_dass21_train = y_dass21[train_indices]
  y_dass21_test = y_dass21[test_indices]

  # Data imputation
  X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                    imputation_method='mean')

  # Classification
  clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_rawdata_train, y_dass21_train)
  d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
  d_preds['preds'] += [ np.array(val) for val in clf.predict(X_rawdata_test) ]
  d_preds['preds_proba'] += [ np.array(val) for val in clf.predict_proba(X_rawdata_test) ]

print('Gradient Boosting')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], d_preds['preds'], average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

Gradient Boosting
Macro ROC AUC One-vs-Rest:  0.4946
Macro ROC AUC One-vs-One:  0.4933
Macro Precision:  0.1935  Recall:  0.2182  F1-score:  0.1947


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Catboost

In [17]:
# Catboost
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

loo = leave_one_person_out(patient_ids)

for patient_id in loo:
  # Leave one out
  train_indices = np.array(loo[patient_id][0])
  test_indices = np.array(loo[patient_id][1])

  X_rawdata_train = X_rawdata[train_indices]
  X_rawdata_test = X_rawdata[test_indices]
  y_dass21_train = y_dass21[train_indices]
  y_dass21_test = y_dass21[test_indices]

  # Data imputation
  X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                    imputation_method='mean')

  # Classification
  clf = CatBoostClassifier(iterations=2, learning_rate=1, depth=2)
  clf.fit(X_rawdata_train, y_dass21_train, verbose=False)
  d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
  d_preds['preds'] += [ np.array(val) for val in clf.predict(X_rawdata_test) ]
  d_preds['preds_proba'] += [ np.array(val) for val in clf.predict_proba(X_rawdata_test) ]

print('Catboost')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], d_preds['preds'], average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

Catboost
Macro ROC AUC One-vs-Rest:  0.4463
Macro ROC AUC One-vs-One:  0.4084
Macro Precision:  0.1558  Recall:  0.1737  F1-score:  0.159


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### LightGBM

In [18]:
# LightGBM
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

loo = leave_one_person_out(patient_ids)

for patient_id in loo:
  # Leave one out
  train_indices = np.array(loo[patient_id][0])
  test_indices = np.array(loo[patient_id][1])

  X_rawdata_train = X_rawdata[train_indices]
  X_rawdata_test = X_rawdata[test_indices]
  y_dass21_train = y_dass21[train_indices]
  y_dass21_test = y_dass21[test_indices]

  # Data imputation
  X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                    imputation_method='mean')

  # Classification
  train_data = lgb.Dataset(X_rawdata_train, label=y_dass21_train)
  test_data = lgb.Dataset(X_rawdata_test, label=y_dass21_test, reference=train_data)
  params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': 6,
        'num_leaves': 31,
        #'learning_rate': 0.05,
        'learning_rate': 0.1,
        'n_estimators': 100,
        'verbose': -1
  }
  bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
  y_dass21_test_preds = bst.predict(X_rawdata_test, num_iteration=bst.best_iteration)
  d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
  d_preds['preds_proba'] += [ np.array(val) for val in y_dass21_test_preds ]

print('LightGBM')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], np.argmax(d_preds['preds_proba'], axis=1), average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM
Macro ROC AUC One-vs-Rest:  0.633
Macro ROC AUC One-vs-One:  0.5244
Macro Precision:  0.1636  Recall:  0.1742  F1-score:  0.1679


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM

In [19]:
# SVM
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

loo = leave_one_person_out(patient_ids)

for patient_id in loo:
  # Leave one out
  train_indices = np.array(loo[patient_id][0])
  test_indices = np.array(loo[patient_id][1])

  X_rawdata_train = X_rawdata[train_indices]
  X_rawdata_test = X_rawdata[test_indices]
  y_dass21_train = y_dass21[train_indices]
  y_dass21_test = y_dass21[test_indices]

  # Data imputation
  X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                    imputation_method='mean')

  # Classification
  clf = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
  clf.fit(X_rawdata_train, y_dass21_train)
  d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
  d_preds['preds'] += [ np.array(val) for val in clf.predict(X_rawdata_test) ]
  d_preds['preds_proba'] += [ np.array(val) for val in clf.predict_proba(X_rawdata_test) ]

print('SVM')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], d_preds['preds'], average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

SVM
Macro ROC AUC One-vs-Rest:  0.4466
Macro ROC AUC One-vs-One:  0.4344
Macro Precision:  0.1335  Recall:  0.1552  F1-score:  0.1429


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random forest

In [20]:
# Random forest
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

loo = leave_one_person_out(patient_ids)

for patient_id in loo:
  # Leave one out
  train_indices = np.array(loo[patient_id][0])
  test_indices = np.array(loo[patient_id][1])

  X_rawdata_train = X_rawdata[train_indices]
  X_rawdata_test = X_rawdata[test_indices]
  y_dass21_train = y_dass21[train_indices]
  y_dass21_test = y_dass21[test_indices]

  # Data imputation
  X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                    imputation_method='mean')

  # Classification
  clf = RandomForestClassifier(max_depth=2, random_state=0)
  clf.fit(X_rawdata_train, y_dass21_train)
  d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
  d_preds['preds'] += [ np.array(val) for val in clf.predict(X_rawdata_test) ]
  d_preds['preds_proba'] += [ np.array(val) for val in clf.predict_proba(X_rawdata_test) ]

print('Random forest')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], d_preds['preds'], average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

Random forest
Macro ROC AUC One-vs-Rest:  0.3352
Macro ROC AUC One-vs-One:  0.3007
Macro Precision:  0.1317  Recall:  0.1667  F1-score:  0.1472


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### MLP

## K-fold

### Gradient Boosting

In [22]:
# Gradient Boosting
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

for seed in range(10):
  kfold = k_fold(patient_ids, number_of_folds=5)
  for k in kfold:
    # K-fold
    train_indices = np.array(kfold[k][0])
    test_indices = np.array(kfold[k][1])

    X_rawdata_train = X_rawdata[train_indices]
    X_rawdata_test = X_rawdata[test_indices]
    y_dass21_train = y_dass21[train_indices]
    y_dass21_test = y_dass21[test_indices]

    # Dummy for random shuffling in k-fold
    if len(set(y_dass21_train)) != len(set(y_dass21)) or len(set(y_dass21_test)) != len(set(y_dass21)):
      continue

    # Data imputation
    X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                      imputation_method='mean')

    # Classification
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                    max_depth=1, random_state=0)
    clf.fit(X_rawdata_train, y_dass21_train)
    d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
    d_preds['preds'] += [ np.array(val) for val in clf.predict(X_rawdata_test) ]
    d_preds['preds_proba'] += [ np.array(val) for val in clf.predict_proba(X_rawdata_test) ]

    # Dummy
    if len(d_preds['preds_proba'][-1]) != len(set(y_dass21)):
      d_preds['labels'].pop(-1)
      d_preds['preds'].pop(-1)
      d_preds['preds_proba'].pop(-1)


print('Gradient Boosting')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], d_preds['preds'], average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

Gradient Boosting
Macro ROC AUC One-vs-Rest:  0.5418
Macro ROC AUC One-vs-One:  0.5507
Macro Precision:  0.2314  Recall:  0.2347  F1-score:  0.2224


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Catboost

In [24]:
# Catboost
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

for seed in range(10):
  kfold = k_fold(patient_ids, number_of_folds=5)
  for k in kfold:
    # K-fold
    train_indices = np.array(kfold[k][0])
    test_indices = np.array(kfold[k][1])

    X_rawdata_train = X_rawdata[train_indices]
    X_rawdata_test = X_rawdata[test_indices]
    y_dass21_train = y_dass21[train_indices]
    y_dass21_test = y_dass21[test_indices]

    # Dummy for random shuffling in k-fold
    if len(set(y_dass21_train)) != len(set(y_dass21)) or len(set(y_dass21_test)) != len(set(y_dass21)):
      continue

    # Data imputation
    X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                      imputation_method='mean')

    # Classification
    clf = CatBoostClassifier(iterations=2, learning_rate=1, depth=2)
    clf.fit(X_rawdata_train, y_dass21_train, verbose=False)
    d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
    d_preds['preds'] += [ np.array(val) for val in clf.predict(X_rawdata_test) ]
    d_preds['preds_proba'] += [ np.array(val) for val in clf.predict_proba(X_rawdata_test) ]

    # Dummy
    if len(d_preds['preds_proba'][-1]) != len(set(y_dass21)):
      d_preds['labels'].pop(-1)
      d_preds['preds'].pop(-1)
      d_preds['preds_proba'].pop(-1)


print('Catboost')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], d_preds['preds'], average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

Catboost
Macro ROC AUC One-vs-Rest:  0.5315
Macro ROC AUC One-vs-One:  0.4474
Macro Precision:  0.1401  Recall:  0.1476  F1-score:  0.1416


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### LightGBM

In [25]:
# LightGBM
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

for seed in range(10):
  kfold = k_fold(patient_ids, number_of_folds=5)
  for k in kfold:
    # K-fold
    train_indices = np.array(kfold[k][0])
    test_indices = np.array(kfold[k][1])

    X_rawdata_train = X_rawdata[train_indices]
    X_rawdata_test = X_rawdata[test_indices]
    y_dass21_train = y_dass21[train_indices]
    y_dass21_test = y_dass21[test_indices]

    # Dummy for random shuffling in k-fold
    if len(set(y_dass21_train)) != len(set(y_dass21)) or len(set(y_dass21_test)) != len(set(y_dass21)):
      continue

    # Data imputation
    X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                      imputation_method='mean')

    # Classification
    train_data = lgb.Dataset(X_rawdata_train, label=y_dass21_train)
    test_data = lgb.Dataset(X_rawdata_test, label=y_dass21_test, reference=train_data)
    params = {
          'objective': 'multiclass',
          'metric': 'multi_logloss',
          'num_class': 6,
          'num_leaves': 31,
          #'learning_rate': 0.05,
          'learning_rate': 0.1,
          'n_estimators': 100,
          'verbose': -1
    }
    bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    y_dass21_test_preds = bst.predict(X_rawdata_test, num_iteration=bst.best_iteration)
    d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
    d_preds['preds_proba'] += [ np.array(val) for val in y_dass21_test_preds ]

    # Dummy
    if len(d_preds['preds_proba'][-1]) != len(set(y_dass21)):
      d_preds['labels'].pop(-1)
      #d_preds['preds'].pop(-1)
      d_preds['preds_proba'].pop(-1)


print('LightGBM')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], np.argmax(d_preds['preds_proba'], axis=1), average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM
Macro ROC AUC One-vs-Rest:  0.6412
Macro ROC AUC One-vs-One:  0.5585
Macro Precision:  0.1778  Recall:  0.2031  F1-score:  0.1885


### SVM

In [29]:
# SVM
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

for seed in range(10):
  kfold = k_fold(patient_ids, number_of_folds=5)
  for k in kfold:
    # K-fold
    train_indices = np.array(kfold[k][0])
    test_indices = np.array(kfold[k][1])

    X_rawdata_train = X_rawdata[train_indices]
    X_rawdata_test = X_rawdata[test_indices]
    y_dass21_train = y_dass21[train_indices]
    y_dass21_test = y_dass21[test_indices]

    # Dummy for random shuffling in k-fold
    if len(set(y_dass21_train)) != len(set(y_dass21)) or len(set(y_dass21_test)) != len(set(y_dass21)):
      continue

    # Data imputation
    X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                      imputation_method='mean')

    # Classification
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
    clf.fit(X_rawdata_train, y_dass21_train)
    d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
    d_preds['preds'] += [ np.array(val) for val in clf.predict(X_rawdata_test) ]
    d_preds['preds_proba'] += [ np.array(val) for val in clf.predict_proba(X_rawdata_test) ]

    # Dummy
    if len(d_preds['preds_proba'][-1]) != len(set(y_dass21)):
      d_preds['labels'].pop(-1)
      d_preds['preds'].pop(-1)
      d_preds['preds_proba'].pop(-1)


print('SVM')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], d_preds['preds'], average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

SVM
Macro ROC AUC One-vs-Rest:  0.5022
Macro ROC AUC One-vs-One:  0.4755
Macro Precision:  0.1525  Recall:  0.1705  F1-score:  0.1596


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random forest

In [30]:
# Random forest
d_preds = {'labels': [], 'preds_proba': [], 'preds': []}

for seed in range(10):
  kfold = k_fold(patient_ids, number_of_folds=5)
  for k in kfold:
    # K-fold
    train_indices = np.array(kfold[k][0])
    test_indices = np.array(kfold[k][1])

    X_rawdata_train = X_rawdata[train_indices]
    X_rawdata_test = X_rawdata[test_indices]
    y_dass21_train = y_dass21[train_indices]
    y_dass21_test = y_dass21[test_indices]

    # Dummy for random shuffling in k-fold
    if len(set(y_dass21_train)) != len(set(y_dass21)) or len(set(y_dass21_test)) != len(set(y_dass21)):
      continue

    # Data imputation
    X_rawdata_train, X_rawdata_test = data_imputation(X_rawdata_train, X_rawdata_test,
                                                      imputation_method='mean')

    # Classification
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_rawdata_train, y_dass21_train)
    d_preds['labels'] += [ np.array(val) for val in y_dass21_test ]
    d_preds['preds'] += [ np.array(val) for val in clf.predict(X_rawdata_test) ]
    d_preds['preds_proba'] += [ np.array(val) for val in clf.predict_proba(X_rawdata_test) ]


print('Random forest')
print('Macro ROC AUC One-vs-Rest: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovr', average='macro'), 4))
print('Macro ROC AUC One-vs-One: ', round(roc_auc_score(d_preds['labels'], d_preds['preds_proba'], multi_class='ovo', average='macro'), 4))
pre, rec, fscore, _ = precision_recall_fscore_support(d_preds['labels'], d_preds['preds'], average='macro')
pre, rec, fscore = round(pre, 4), round(rec, 4), round(fscore, 4)
print('Macro Precision: ', pre, ' Recall: ', rec, ' F1-score: ', fscore)

Random forest
Macro ROC AUC One-vs-Rest:  0.3857
Macro ROC AUC One-vs-One:  0.3615
Macro Precision:  0.2896  Recall:  0.1711  F1-score:  0.1502


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
